In [1]:
#Standard Libraries and pyxem + hyperspy
%matplotlib qt
import numpy as np
import math
import hyperspy.api as hs
import pyxem as pxm
import diffpy.structure
from matplotlib import pyplot as plt

#Pyxem sub-libraries
from diffsims.generators.structure_library_generator import StructureLibraryGenerator
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator, VectorLibraryGenerator
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.generators.indexation_generator import VectorIndexationGenerator
from pyxem.utils.sim_utils import sim_as_signal
from pyxem.utils.indexation_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks

#For testing pyxem functions in-dept:
from transforms3d.euler import euler2quat, quat2axangle, euler2axangle
from pyxem.signals import transfer_navigation_axes_to_signal_axes
from diffsims.utils.vector_utils import get_angle_cartesian

#For generating the correct implementation of the get_orientations_from_stereographic_triangle() function:
from itertools import product
from transforms3d.euler import axangle2euler
from scipy.spatial.transform import Rotation as R

#For plotting
from orix.quaternion.orientation import Orientation, Misorientation
from orix.quaternion.symmetry import O#,C1, D6h,Oh
from orix.quaternion.orientation_region import OrientationRegion
from orix import plot

Defining the function to create candidate orientations and replace get_orientations_from_stereographic_triangle()

In [ ]:
def create_linearly_spaced_array_in_rzxz(resolution):
    """
    If covering whole domain we would use angular ranges alpha [0,360], beta [0,180] and gamma [0,360] in
    line with Convention 4 described in Reference [1]. For space group 215 we have a 4 fold axis parrallel
    to z, so we can drop the alpha and gamma ranges to 90, saving a bit of time
    
    
    References
    ----------
    [1]  D Rowenhorst et al 2015 Modelling Simul. Mater. Sci. Eng.23 083501
         https://iopscience.iop.org/article/10.1088/0965-0393/23/8/083501/meta
    """
    #Can invert to save time, and use 0,360 for alpha.
    num_steps = int(180/resolution + 0.5)
    alpha = np.linspace(0,360,num=int(num_steps*2),endpoint=False)
    beta  = np.linspace(0,180,num=int(num_steps),endpoint=False)
    gamma = np.linspace(0,90,num=int(num_steps/2),endpoint=False)
    z = np.asarray(list(product(alpha, beta, gamma)))
    good_eggs = []
    for i in range(z.shape[0]):
        if(z[1] is 0):
            if(z[2] is 0):
                good_eggs.append(i)
        else:
            good_eggs.append(i)
               
    return z[good_eggs,:]


def convert_axangle_to_correct_range(vector,angle): 
    if (angle >= 0) and (angle < np.pi): #input in the desired convention
        pass
    elif (angle >= -np.pi) and (angle < 0):
        vector = np.multiply(vector,-1)
        angle  = angle * -1
    elif (angle >= np.pi) and (angle < 2*np.pi):
        vector = np.multiply(vector,-1)
        angle = 2*np.pi - angle
    else:
        raise ValueError("You have an axis-angle angle outside of acceptable ranges")

    return vector,angle


def convert_rzxz_array_to_axangle(z):
    stored_axangle = np.ones((z.shape[0],4))
    z = np.deg2rad(z) #for the transform operation
    for i,row in enumerate(z):
        temp_vect, temp_angle = euler2axangle(row[0],row[1],row[2],'rzxz')
        temp_vect,temp_angle  = convert_axangle_to_correct_range(temp_vect,temp_angle)
        for j in [0,1,2]:
            stored_axangle[i,j] = temp_vect[j]
            stored_axangle[i,3] = temp_angle #in radians!

    return stored_axangle


def convert_axangle_to_euler(z):
    stored_euler = np.ones((z.shape[0],3))
    for i,row in enumerate(z):
        a_array = axangle2euler(row[:3],row[3],'rzxz')
        for j in [0,1,2]:
            stored_euler[i,j] = a_array[j]

    stored_euler = np.rad2deg(stored_euler)
    return stored_euler

def get_orientations_from_stereographic_triangle_new(resolution):
    #Create lineary spaced function in cubic symmetry domain
    rzxz_array = create_linearly_spaced_array_in_rzxz(resolution)
    
    #Convert to axangle.
    axangle_array = convert_rzxz_array_to_axangle(rzxz_array)
    
    axangle_array = axangle_array[axangle_array[:,3] < np.deg2rad(62.8)]
    rf_array = axangle_array.copy()
    rf_array[:,3] = np.tan(np.divide(rf_array[:,3],2))
    tanangle = rf_array[:,3]
    rf_array[:,:3] = np.multiply(rf_array[:,:3],tanangle.reshape(rf_array.shape[0],1))
    
    for normal_vector in [[1,1,1],[1,1,-1],[1,-1,-1],[1,-1,1],[-1,-1,-1],[-1,-1,1],[-1,1,1],[-1,1,-1]]:
        temporary_truth = np.abs(np.dot(rf_array[:,:3],np.divide(normal_vector,np.sqrt(3)))) < np.sqrt(1/3)#np.sqrt(1/3) 
        rf_array = rf_array[temporary_truth]
        axangle_array = axangle_array[temporary_truth] #saves us some conversion later on
        
    final_euler_array = convert_axangle_to_euler(axangle_array)
    rot_list = []
    for row in final_euler_array:
        rot_list.append((row[0],row[1],row[2]))
        
    return rot_list

Define a gold structure, could also be done using a .cif file.

In [2]:
latt = diffpy.structure.lattice.Lattice(4.08, 4.08, 4.08, 90, 90, 90)
atom = diffpy.structure.atom.Atom(atype='Au', xyz=[0,0,0], lattice=latt)
atom2 = diffpy.structure.atom.Atom(atype='Au', xyz=[0,1/2,1/2], lattice=latt)
atom3 = diffpy.structure.atom.Atom(atype='Au', xyz=[1/2,0,1/2], lattice=latt)
atom4 = diffpy.structure.atom.Atom(atype='Au', xyz=[1/2,1/2,0], lattice=latt)
au = diffpy.structure.Structure(atoms=[atom,atom2,atom3,atom4], lattice=latt)

Define simulation parameters.

In [3]:
pattern_size = 256  # pixels
half_pattern_size = pattern_size // 2
reciprocal_radius = 4 #How far out in reciprocal space we look for reflections
calibration = reciprocal_radius / half_pattern_size #Scaling of diffraction pattern
beam_energy = 300.0 #keV

ediff = DiffractionGenerator(beam_energy, 0.025)  # keV and relrod length (1/Å)

Simulate gold diffraction patterns in a set of orientations

In [4]:
phase_name = ['Au'] 
Angles = [] #Not the ones flying in the sky
discreteAngleValues = np.linspace(-60.,60.,25)
lengthOfAnglesVek = len(discreteAngleValues)
for i in range (lengthOfAnglesVek):#(int(lengthOfAnglesVek/2)+1):
    for j in range (lengthOfAnglesVek):
        Angles.append((float(discreteAngleValues[j]),float(discreteAngleValues[i]),float(0)))

sample_lib = StructureLibrary(phase_name, [au], [Angles])

diff_gen = DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(sample_lib,
                                           calibration=calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=(half_pattern_size, half_pattern_size),
                                           with_direct_beam=False)

data_au = []

for angles in Angles:
    pattern = sim_as_signal(library.get_library_entry(phase='Au', angle=(angles))['Sim'],
                            pattern_size, 0.05, reciprocal_radius)
                            
    data_au.append(pattern)
        
data = [x.data for x in data_au] 

test_data = pxm.ElectronDiffraction2D(np.asarray(data).reshape(lengthOfAnglesVek,lengthOfAnglesVek, pattern_size, pattern_size))
test_data.set_diffraction_calibration(calibration) 

Generate a library, either by the use of an orientation list, or by the use of the stereographic triangle function

In [5]:
structure_library_generator = StructureLibraryGenerator(
    [('Au', au, 'cubic')])

In [ ]:
orientation_list = get_orientations_from_stereographic_triangle_new(1.4)
structure_library = structure_library_generator.get_orientations_from_list([orientation_list])

In [ ]:
structure_library = structure_library_generator.get_orientations_from_stereographic_triangle([[0,]],1)

In [6]:
structure_library = structure_library_generator.get_orientations_from_list([Angles])

Simulate diffraction patterns and store in a template library

In [7]:
diff_gen = DiffractionLibraryGenerator(ediff)
template_library = diff_gen.get_diffraction_library(structure_library,
                                                    calibration=calibration,
                                                    reciprocal_radius=reciprocal_radius,
                                                    half_shape=(half_pattern_size, half_pattern_size),
                                                    with_direct_beam=False)

Find best matching template for every pixel of the simulated data

In [8]:
indexer = IndexationGenerator(test_data, template_library)
match_results = indexer.correlate(n_largest=3)

Plot best matching template on top of the experimental diffraction pattern

In [ ]:
match_results.plot_best_matching_results_on_signal(test_data, template_library, permanent_markers=False, cmap='magma_r')

Built in method for generating an orientation map

In [9]:
cryst_map = match_results.get_crystallographic_map()
ori_map = cryst_map.get_orientation_map()
ori_map.plot(cmap='inferno',axes_off = True,scalebar = False)

Plot a template from the template library. 

In [ ]:
library_entries = template_library.get_library_entry(phase = 'Au', angle = (-60,-60,0))#(100,140,0)

intensities = library_entries['intensities']
Coordinates = library_entries['pixel_coords']

PlotOfIntensities = np.zeros((pattern_size,pattern_size))
for i in range (len(intensities)):
    if(Coordinates[i,0]>0 and Coordinates[i,0] < pattern_size and Coordinates[i,1]>0 and Coordinates[i,1] < pattern_size ):
        PlotOfIntensities[Coordinates[i,1],Coordinates[i,0]]=1
    
fig, axs = plt.subplots(1, 1, figsize=(6, 6))
axs.imshow(PlotOfIntensities, cmap = 'inferno')
axs.set_title('Euler Angle: ({0:.0f},{1:.0f},{2:.0f})'.format(-60,-60,0))

Plot Correlation scores

In [ ]:
correlation_score_array = np.zeros((25,25))
for i in range (match_results.data[:,:,0,2].shape[0]):
    for j in range (match_results.data[:,:,0,2].shape[1]):
        correlation_score_array[i,j] = float(match_results.data[i,j,0,2])

fig, (ax1) = plt.subplots(1, 1, figsize=(15, 6))
im =  ax1.imshow(correlation_score_array,cmap = 'inferno',vmin = 0)
ax1.set_title('Correlation score')
ax1.set_xticks(ticks = np.linspace(0,24,num = 7))
ax1.set_xticklabels(np.linspace(-60,60,7))
ax1.set_yticks(ticks = np.linspace(0,24,num = 7))
ax1.set_yticklabels(np.linspace(-60,60,7))
ax1.set_xlabel('$\phi_1$')
ax1.set_ylabel('$ \Theta $')
fig.colorbar(im, ax = ax1)
plt.show()

Generate orientation and misorientation plots.

In [ ]:
array = match_results.data[:,:,0,1]
b = []
list1 = array.tolist()
numberInList = list1[0][0][0]
for i in range (len(array)):
    for j in range ( len(list1[i])):
        b.append([list1[i][j][0]+np.finfo(type(numberInList)).eps* 3,list1[i][j][1]+
                  np.finfo(type(numberInList)).eps * 3, list1[i][j][2]]+np.finfo(type(numberInList)).eps * 3)


In [ ]:
ZeroAngle = []
list1 = array.tolist()
for i in range (len(list1)):
    for j in range ( len(list1[i])):
        ZeroAngle.append([0,0,0])

In [ ]:
ori = Orientation.from_euler(np.radians(b)).reshape(25,25).set_symmetry(O) #Match Results
ori2 = Orientation.from_euler(np.radians((ZeroAngle))).reshape(25,25).set_symmetry(O) #All zeros

# Compute misorientations
misori_base = Misorientation(~ori[:, :] * ori2[:, :])
misori = misori_base.set_symmetry(O, O)
a = np.rad2deg(misori.angle.data)


ori3 = Orientation.from_euler(np.radians(b)).reshape(25,25).set_symmetry(O) #Match Resullts
ori4 = Orientation.from_euler(np.radians((Angles))).reshape(25,25).set_symmetry(O) #List of orientations in the simulated diffraction patterns, not template library

# Compute misorientations
misori_base2 = Misorientation(~ori3[:, :] * ori4[:, :])
misori2 = misori_base2.set_symmetry(O, O)

#Convert to degrees
c = np.rad2deg(misori2.angle.data)

#Plot orientation map and misorientation map
fig, [ax1,ax2] = plt.subplots(1, 2, figsize=(15, 6))
mappable= ax1.imshow(a,cmap = 'inferno')
ax1.set_title('Distance to (0,0,0)')
ax1.set_xticks(ticks = np.linspace(0,24,num = 7))
ax1.set_xticklabels(np.linspace(-60,60,7))
ax1.set_yticks(ticks = np.linspace(0,24,num = 7))
ax1.set_yticklabels(np.linspace(-60,60,7))
ax1.set_xlabel('$\phi_1$')
ax1.set_ylabel('$ \Theta $')
mappable2 = ax2.imshow(c,cmap = 'inferno')
ax2.set_title('Template matching result, distance to true value')
ax2.set_xticks(ticks = np.linspace(0,24,num = 7))
ax2.set_xticklabels(np.linspace(-60,60,7))
ax2.set_yticks(ticks = np.linspace(0,24,num = 7))
ax2.set_yticklabels(np.linspace(-60,60,7))
ax2.set_xlabel('$\phi_1$')
ax2.set_ylabel('$ \Theta $')
fig.colorbar(mappable, ax = ax1)
fig.colorbar(mappable2, ax = ax2)
plt.show()